# De-Noising Satellite Images

The project mainly focuses on removing noise from digital images taken from satellites . Our
underlying goal in this project is to estimate the original image by suppressing noise from a
noise-contaminated version of the image . One of the main tasks which needs special attention during
denoising is the complete and quantitative analysis of noise before choosing the best suited filters to
smooth the images .

## 1. Importing Libraries

In [1]:
# THIS CELL IS RESRVERED FOR IMPORTS 

import math
import cv2
import numpy as np
from scipy import stats,misc,ndimage
from scipy.signal import wiener,convolve2d
from skimage.util import img_as_float
from skimage import restoration
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread, imshow, imsave

In [2]:
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askopenfile
from PIL import Image, ImageTk
import os

## 2. Supporting Image Processing Functions

In [3]:
def load(file_path):
    image = imread(file_path)
    return image

def display(image,title="Image"):
    plt.figure(figsize=[10,10])
    channels=len(image.shape)
    if channels<3:
        plt.imshow(image,cmap='Greys_r');
    else:
        plt.imshow(image);      
    plt.title(title);plt.axis("off");        

def grayscale(image):
    channels=len(image.shape)
    if channels<3:
        return img
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=grayscale_image
    return image

def plot_histogram(image):
    colors = ("red", "green", "blue")
    channel_ids = (0, 1, 2)

    plt.figure()
    plt.xlim([0, 256])
    for channel_id, c in zip(channel_ids, colors):
        histogram, bin_edges = np.histogram(
            image[:, :, channel_id], bins=256, range=(0, 256)
        )
        plt.plot(bin_edges[0:-1], histogram, color=c)

    plt.title("Color Histogram")
    plt.xlabel("Color value")
    plt.ylabel("Pixel count")

In [4]:
def max_intensity(img,chanels):
    maxi=np.zeros((chanels))
    for x in range(0,img.shape[0]):
        for y in range(0,img.shape[1]):
            for z in range(0,chanels):
                if chanels==1 and img[x][y]>maxi[z] :
                    maxi[z]=img[x][y]
                elif chanels!=1 and img[x][y][z]>maxi[z]:
                    img[x][y][z]>maxi[z]
                    maxi[z]=img[x][y][z]
    return maxi

def min_intensity(img,chanels):
    mini=np.zeros((chanels))
    for x in range(0,img.shape[0]):
        for y in range(0,img.shape[1]):
            for z in range(0,chanels):
                if chanels==1 and img[x][y]<mini[z] :
                    mini[z]=img[x][y]
                elif chanels!=1 and img[x][y][z]<mini[z] :
                    mini[z]=img[x][y]
    return mini


L=256
def contrast_stretch(img):
    #img -> image can be Grayscale or RGB
    chanels=1
    if len(img.shape)>2:
        chanels=img.shape[2]
    max_i=max_intensity(img,chanels)
    min_i=min_intensity(img,chanels)
    my_value=0
    if chanels==1 :
        stretched_img=np.zeros((img.shape[0],img.shape[1])).astype(int)
    else :
        stretched_img=np.zeros((img.shape[0],img.shape[1],img.shape[2])).astype(int)

    for x in range(0,img.shape[0]):
        for y in range(0,img.shape[1]):
            for z in range(0,chanels):
                if chanels==1:
                    my_value=((img[x][y]-min_i[z])/(max_i[z]-min_i[z]))
                    stretched_img[x][y]=int(my_value*L)
                else :
                    my_value=((img[x][y][z]-min_i[z])/(max_i[z]-min_i[z]))
                    stretched_img[x][y][z]=int(my_value*L)
    stretched_img=stretched_img.astype(int)
    return stretched_img

## 3. Noise Type Detection (Gussian , Salt n Pepper & Speckle)

In [5]:
def noise_type_detection(img):
    noises=[]
    noises.append(saltNpepper_detection(img))
    noises.append(speckle_detection(img))
    noises.append(gussian_detection(img))
    return noises

def gussian_detection(img): 
#  This function tests the null hypothesis that a sample comes from a normal distribution. 
#  It is based on D’Agostino and Pearson’s test that combines skew and kurtosis to produce an omnibus test of normality.

    #Reference :- https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html
    k2, p = stats.normaltest(img)
    alpha = 1e-3
    count=0

    #check if the image has the normal distribution 
    for x in range(img.shape[1]):
        if p[x].all() >alpha:
            count+=1 
            
    if count>img.shape[1]*0.95:
        return 1
    return 0

def saltNpepper_detection(img):
    ones=[0,0,0]
    zeros=[0,0,0]
    for x in range(0,3):
        ones[x] = np.count_nonzero(img == 255-x)
        
    for x in range(0,3):
        zeros[x] = np.count_nonzero(img == x) 
    
    avg_low = np.average(zeros)
    avg_high = np.average(ones)

    max_low=np.max(zeros)
    max_high=np.max(ones)
    
    first=max(avg_low,max_low)
    second=max(avg_high,max_high)

    percentage_salt=(first*100)/(img.shape[0]*img.shape[1])
    percentage_pepper=(second*100)/(img.shape[0]*img.shape[1])
    if (percentage_salt+percentage_pepper)>30:
        return 1
    return 0
  
def speckle_detection(img):
    left_extreme_count=right_extreme_count=0
    for x in range(0,3):
        left_extreme_count+=np.count_nonzero(img == 255-x)
        
    for x in range(0,3):
        right_extreme_count+=np.count_nonzero(img == x)
    
    total_pixels=img.shape[0]*img.shape[1]
    if ((total_pixels-left_extreme_count-right_extreme_count)*100/total_pixels)<40:
        return 1
    return 0

## 4. Spatial Domain Filtering (Median , Bilateral , Gussian , Wiener) 

In [6]:
def gausskernel(size,k=1,sigma=1):
    gausskernel = np.zeros((size,size),np.float32)
    for i in range (size):
        for j in range (size):
            norm = math.pow(i-k,2) + pow(j-k,2)
            gausskernel[i,j] = math.exp(-norm/(2*math.pow(sigma,2)))/2*math.pi*pow(sigma,2)
    sum = np.sum(gausskernel)
    kernel = gausskernel/sum 
    return kernel

def gussian_filtering(img_gray):
    k=1
    size = 2*k+1
    h,w = img_gray.shape
    return cv2.GaussianBlur(img_gray,(9,9),cv2.BORDER_DEFAULT)
    kernel = gausskernel(size,k,1.5)
    
    k_h,k_w = kernel.shape
    
    for i in range(int(k_h/2),h-int(k_h/2)):
        for j in range(int(k_h/2),w-int(k_h/2)):
            sum = 0
            for k in range(0,k_h):
                for l in range(0,k_h):
                    sum += img_gray[i-int(k_h/2)+k,j-int(k_h/2)+l]*kernel[k,l]
            img_gray[i,j] = sum
    return img_gray

def median_filtering(img,kernel=9):
    median_blurred = cv2.medianBlur(img,kernel)
    return median_blurred

def weiner_filtering(img):
    psf = np.ones((5,5)) / 25
    img1 = convolve2d(img,psf,'same')
    img1 += 0.1 * img1.std() * np.random.standard_normal(img1.shape)
    wiener_filtered = restoration.wiener(img1,psf,11)
    return wiener_filtered

def spatial_filtering(img):
    if len(img.shape)>2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    noises=noise_type_detection(img)

    if noises[0]==1:
        median_filtering(img)
    if noises[1]==1:
        weiner_filtering(img)
    img = cv2.bilateralFilter(img,20,20,20) 
#     blurred_f = ndimage.gaussian_filter(img, 5)
#     filter_blurred_f = ndimage.gaussian_filter(blurred_f, 1)
#     alpha =5
#     sharpened = blurred_f + alpha * (blurred_f - filter_blurred_f)
    final_image = cv2.equalizeHist(img)

    return final_image

## 5. Frequency Domain Filtering (Low Pass , High Pass & etc.) 

In [7]:
def lpass_frequency_filtering(img):
    kernel_size = 5
    lowpass_kernel_gaussian = gausskernel(kernel_size)
    lowpass_kernel_gaussian = lowpass_kernel_gaussian / lowpass_kernel_gaussian.sum()

    lowpass_kernel_box = np.ones((kernel_size, kernel_size))
    lowpass_kernel_box = lowpass_kernel_box / (kernel_size * kernel_size)

    lowpass_image_gaussian = cv2.filter2D(img, -1, lowpass_kernel_gaussian)
    lowpass_image_box = cv2.filter2D(img, -1, lowpass_kernel_box)
    return lowpass_image_gaussian,lowpass_image_box

def hpass_frequency_filtering(img):
    kernel_size = 5
    highpass_image_gaussian = img - lpass_frequency_filtering(img)[0]
    highpass_image_gaussian = np.absolute(highpass_image_gaussian)

    highpass_image_box = img - lpass_frequency_filtering(img)[1]
    highpass_image_box = np.absolute(highpass_image_box)
    return highpass_image_box

def band_reject_frequency__filtering(img):
    return hpass_frequency_filtering(img)+lpass_frequency_filtering(img)

def band_pass_frequency__filtering(img):
    bandpass_image = img - band_reject_frequency__filtering(img)
    bandpass_image = np.absolute(bandpass_image)
    return bandpass_image

def frequency_filtering(img):
    x1=lpass_frequency_filtering(img)[0]
    x2=hpass_frequency_filtering(img)[0]
    if len(x1.shape)>2:
        img = cv2.cvtColor(x1, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(clipLimit=4.5, tileGridSize=(8, 8))
    final = clahe.apply(img)
    return final

## 6.Tkinter GUI Code 

In [8]:
def predict(filepath):
    original=load(filepath)
    corrected=''
    global clicked
    if clicked.get()=="Spatial Domain Filtering                                ":
        corrected=spatial_filtering(load(filepath))
    else :
        corrected=frequency_filtering(load(filepath))    
    current=os.getcwd()+'\\output.jpg'
    imsave("output.jpg",corrected, cmap='gray')
    
    current=os.getcwd()+'\\output.jpg'
    img1=Image.open(current) # read the image file
    img1=img1.resize((240,240)) # new width & height
    img1=ImageTk.PhotoImage(img1)
    e1 =Label(image_gui1)
    e1.pack()
    e1.image = img1 # keep a reference! by attaching it to a widget attribute
    e1['image']=img1 # Show Image 
    b1 = Button(image_gui1, text='Make Another Correction -->',width=25,command = lambda:destroy())
    b1.pack(anchor="e", padx=10, pady=10)
#         result.config(font=('Helvetica bold',15))
#         result.pack()
#     else :
#         result=Label(text="Defected")
#         result.config(font=('Helvetica bold',15))
#         result.pack()

#     timr=Label(text=elapsed_time)
#     timr.config(font=('Helvetica bold',7))
#     timr.pack()

def destroy():
    global image_gui1
    image_gui1.destroy()
    image_gui1= Tk()
    intial_window()

In [9]:
def upload_file(image_gui1):
    global img
    f_types = [('jpg Files', '*.jpg')]
    global filename
    filename = filedialog.askopenfilename(filetypes=f_types)
    img=Image.open(filename) # read the image file
    img=img.resize((240,240)) # new width & height
    img=ImageTk.PhotoImage(img)
    e1 =Label(image_gui1)
    e1.pack()
    e1.image = img # keep a reference! by attaching it to a widget attribute
    e1['image']=img # Show Image 
    b1 = Button(image_gui1, text='Apply Correction',width=20,command = lambda:predict(filename))
    b1.pack()
    
def print_sapce(x):
    for y in range(0,x):
        my_space=Label(text="")
        my_space.pack()
        
def print_text(x):
    my_text=Label(text=x)
    my_text.pack()

In [10]:
def intial_window():

    image_gui1.geometry("900x900")  # Size of the window 
    image_gui1.iconbitmap(r"logo.ico")

    image_gui1.title('Satellite Images De-Noising')
    my_font1=('times', 18, 'bold')
    
    company_label=Label(text="Demonstration for De-Noising Satellite Images")
    company_label.config(font=('Helvetica bold',15))
    company_label.pack()

    
    image = Image.open('logo.png').resize((100, 100))
    my_logo = ImageTk.PhotoImage(image)
    lgo_label=Label(image=my_logo)
    lgo_label.pack()
    
    
    
    options = [
        "Spatial Domain Filtering                                ",
        "Frequency Domain Filtering                              "
    ]
    global clicked 
    clicked= StringVar()

    print_sapce(1)
    clicked.set( "Select type of de-noising you want to apply ; " )
    drop = OptionMenu( image_gui1 , clicked , *options )
    drop.pack()
        
    b1 = Button(image_gui1, text='Upload Image',width=20,command = lambda:upload_file(image_gui1))
    b1.pack()
    print_sapce(1)

    image_gui1.mainloop()

### Main Driver of the Satellite Images De-Noiser

In [11]:
image_gui1= Tk()
intial_window()